In [1]:
import numpy as np
import pandas as pd
import time
import re

# Função Verificadora de CPF

In [2]:
def validador_de_cpf(CPF):
    ###Separar digitos do CPF
    CPF =str(CPF)                                         #Transforma em uma string para poder acessar cada posição
    CPF = re.sub(r'\D', '', CPF)                          #Retira todos os caracteres que não são dígitos ('D') por vazio
    cpf = np.arange(0, len(CPF))                          #Cria um vetor 
    for i in range(len(CPF)):                             #Adiciona cada dígito inteiro ao vetor criado
        cpf[i] = int(CPF[i])
        
    ###Primeira Verificação
    if len(cpf) !=11:                                     #Verifica se o CPF tem 11 dígitos
        return False
    
    ###Obtenção do primeiro dígito verificador
    peso1 = np.array([10, 9, 8, 7, 6, 5, 4, 3, 2])       #Pesos utilizados na verificação do primeiro dígito verificador
    verificação = sum(cpf[0:9] * peso1)                  #Verificação: digitos do cpf multiplicados pelos pesos, seguido pela soma
    
    if verificação % 11 == 0:                            #Se resto da divisão por 11 for igual a 0, primeiro digito é igual a 0
        digito1 = 0
    else:                                                #Se for diferente de 0, primeiro digito é igual a 11 menos o resto da divisão
        digito1 = 11 - (verificação % 11)
    
    ###Obtenção do segundo dígito verificador
    peso2 = np.array([11, 10, 9, 8, 7, 6, 5, 4, 3, 2])  #Pesos utilizados na verificação do primeiro dígito verificador
    verificação = sum(cpf[0:10] * peso2) 
                                              
    digito2 = 11 - (verificação % 11)
    
    ###Verificação dos dois dígitos obtidos com os dois dígitos do CPF
    if digito1 != cpf[9] or digito2 != cpf[10]:
        return False
    else:
        return True

# Função Otimizada

In [3]:
def validador_de_cpf_otimizado(CPF):
    cpf = np.array([int(i) for i in str(CPF) if i.isdigit()])       #Compreensão de lista para transformar a string do CPF em um array de inteiros, avaliando se todos os caracteres são números
    
    ###Primeira Verificação
    if len(cpf) !=11:                                               #Verifica se o CPF tem 11 dígitos
        return False
    
    ###Pesos
    peso1 = np.array([10, 9, 8, 7, 6, 5, 4, 3, 2])
    peso2 = np.array([11, 10, 9, 8, 7, 6, 5, 4, 3, 2])
    
    ##Obtenção do primeiro dígito
    verificação = np.sum(cpf[:9] * peso1)
    if verificação % 11 == 0:                                      #Se resto da divisão por 11 for igual a 0, primeiro digito é igual a 0
        digito1 = 0
    else:                                                          #Se for diferente de 0, primeiro digito é igual a 11 menos o resto da divisão
        digito1 = 11 - (verificação % 11)
        
    ##Primeira comparação de dígitos                               #Realização da comparação do primeiro dígito obtido com o fornecido. Caso seja diferente, já finaliza a execução
    if digito1 != cpf[9]:
        return False
    
    ##Obtenção do segundo dígito
    verificação = np.sum(cpf[:10] * peso2)                      
    digito2 = 11 - (verificação % 11)
    
    ##Primeira comparação de dígitos
    if digito2 != cpf[10]:
        return False
    else:
        return True
    

# Função para medir tempo de execução

In [7]:
###Função Para medir tempo
def medir_tempo(funcao, numeros):                                    #Tem como parâmetros a função a ser utilizada e o(s) CPF(s) a ser(em) validado(s) 
    tempos = []                                                      #Cria uma lista vazia para adicionar os tempos de execução
    lista = []
    for cpf in numeros:
        inicio = time.time()                                         #Timer inicial
        lista.append([cpf, funcao(cpf)])                             #Constroi a lista com os CPFs e as verificações
        fim = time.time()                                            #Timer final
        tempos.append(fim - inicio)                                  #Adiciona na lista de tempos a diferenta entre o timer final e inicial
    CPF_dt = pd.DataFrame(lista, columns = ['CPF', 'Validação'])     #Cria um DataFrame para guardar o resultado da Verificação dos CPFs 
    
    return np.mean(tempos), CPF_dt                                   #Retorna a média de tempos e o dataframe com as validações realizadas

# Gerar números de 0 a 10_000_000

In [5]:
lista_cpfs = [str(i).zfill(11) for i in range(10_000_001)]      #Utiliza a função zfill para adicionar zeros a esquerda para completar os 11 números

# Comparação de desempenho baseada no tempo de execução

In [9]:
###Execução da função para medição de tempo e validação dos CPFs com a função original 
tempo_validador_de_cpf, validador_de_cpf_dt = medir_tempo(validador_de_cpf, lista_cpfs)

In [19]:
###Pandas DataFrame com o resultado da validação dos CPFs com a função original
validador_de_cpf_dt.head(10000001)

,CPF,Validação
0,00000000000,False
1,00000000001,False
2,00000000002,False
3,00000000003,False
4,00000000004,False
...,...,...
9999996,00009999996,False
9999997,00009999997,False
9999998,00009999998,False
9999999,00009999999,False


In [20]:
###Execução da função para medição de tempo e validação dos CPFs com a função otimizada
tempo_validador_de_cpf_otimizado, validador_de_cpf_otimizado_dt = medir_tempo(validador_de_cpf_otimizado, lista_cpfs)

In [23]:
###Pandas DataFrame com o resultado da validação dos CPFs com a função otimizada
validador_de_cpf_otimizado_dt.head(10000001)

,CPF,Validação
0,00000000000,False
1,00000000001,False
2,00000000002,False
3,00000000003,False
4,00000000004,False
...,...,...
9999996,00009999996,False
9999997,00009999997,False
9999998,00009999998,False
9999999,00009999999,False


In [24]:
print("Resultado da comparação de tempo de execução entre as duas funções")
print("Tempo original: ", tempo_validador_de_cpf)
print("Tempo otimizado: ", tempo_validador_de_cpf_otimizado)

Resultado da comparação de tempo de execução entre as duas funções
Tempo original:  1.7780710921819046e-05
Tempo otimizado:  1.552186410180529e-05
